In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
import pandas as pd 
import numpy as np
import json
from tqdm import tqdm
import torch.nn as nn


2024-02-01 12:52:05.771400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 12:52:05.771498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 12:52:05.931066: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.1.2
CUDA Available: True
CUDA Version: 12.1


In [3]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16).to('cuda')


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# Move the model to the GPU(s)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)




In [6]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [7]:
def generate_text(prompt, tokens, temp):
    torch.cuda.reset_peak_memory_stats
    
    
    # Move model inputs to the GPU(s)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    
    
    # Generate text
    if temp:
        generated_ids = model.generate(**model_inputs, max_new_tokens=tokens, do_sample=True, top_k=10, num_return_sequences=1, temperature=temp)
    else:
        generated_ids = model.generate(**model_inputs, max_new_tokens=tokens, do_sample=True, top_k=10, num_return_sequences=1)
    
    # Decode and clean up
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].replace(prompt, "")
    
    # Remove inputs from GPU
    del model_inputs
    torch.cuda.empty_cache()
    
    return answer

In [8]:
generate_text("LOL", 50, None)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


', I just got home and read your post. I’m a 40 year old male, 6 foot tall and weigh about 180 lbs. I began this diet as a last resort to try and fix a lot of'

In [9]:
def get_mistral_responses(n, df):
    responses = {}
    for i  in range(n):
        responses[i + 1] = []
    for i in tqdm(range(n)):
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
            response = generate_text(row["Prompt"], 100, None)
            responses[i + 1].append(response)

    
    return responses

#### First englisch

In [11]:
df = pd.read_csv('/kaggle/input/prompts-data/prompts.csv')
df

,Scenario,Variation,Prompt
0,Switch,base,Consider following szenario where you are the ...
1,Loop,base,Consider following szenario where you are the ...
2,Footbridge,base,Consider following szenario where you are the ...
3,Switch,apples,Consider following szenario where you are the ...
4,Loop,apples,Consider following szenario where you are the ...
5,Footbridge,apples,Consider following szenario where you are the ...
6,Switch,scientist,Consider following szenario where you are the ...
7,Loop,scientist,Consider following szenario where you are the ...
8,Footbridge,scientist,Consider following szenario where you are the ...
9,Switch,two,Consider following szenario where you are the ...


In [12]:
responses = get_mistral_responses(50, df)
for key, value in responses.items():
    df[f"response_{key}"] = value

Processing Rows:   0%|          | 0/18 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:   6%|▌         | 1/18 [00:05<01:29,  5.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  11%|█         | 2/18 [00:10<01:23,  5.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  17%|█▋        | 3/18 [00:15<01:18,  5.25s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  22%|██▏       | 4/18 [00:20<01:13,  5.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  28%|██▊       | 5/18 [00:26<01:07,  5.19s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  33%|███▎      | 6/18 [00:31<01:02,  5.19s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:  39%|███▉      | 7/18 [00:36<00:57,  5.18s/it]Setting `pad_token_id

In [13]:
df.to_csv("results_mistral.csv")